### 1. Giới thiệu về Long Short Term Memory  (LSTM)

Long Short Term Memory (LSTM) là mạng cải tiến của RNN nhằm khắc phục nhược điểm "nhanh quên" trong mô hình RNN. Như đã trình bày trong bài trước "Mô hình Recurrent Neural Networks", mô hình RNN khá đơn giản nên việc liên kết các từ phía trước nó không được tốt do các vấn đề về Vanishing Gradient. 

Cấu trúc mạng LSTM được giới thiệu bởi [Hochreiter & Schmidhuber (1997)](http://www.bioinf.jku.at/publications/older/2604.pdf), sau đó được nhiều chuyên gia cải tiến thêm và hiện tại LSTM rất hiệu quả trong nhiều bài toán và đang được sử dụng rộng rãi.

Ý tưởng chính của mô hình LSTM:
- Tạo thêm bộ nhớ (memory) để nhớ được thông tin ở xa trước đó.
- Cho phép thông tin được đổ xuống bộ nhớ và hidden units mạnh hoặc yếu khác nhau tuỳ thuộc vào input tại thời điểm dó.

Cấu trúc LSTM gồm 4 thành phần chính: Cell State, Input Gate, Output Gate và Forget Gate.

Các Cell có nhiệm vụ nhớ các giá trị trong khoảng thời gian phụ thuộc vào giá trị của cổng Forget gate.



### 2. Nhắc lại mô hình RNN 

Với mô hình RNN cơ bản, nó có cấu trúc đơn giản và thường chỉ có 1 hàm `tanh` như hình sau:

<img src="images/11_LSTM3-SimpleRNN.png" style="width:70%;height:70%;">

Đối với mô hình này ta có: **$h_t = \sigma({Wx_t + Uh_{t-1}})$**

Xét bài toán sử lý câu sử dụng RNN, đầu tiên, các từ trong câu sẽ được chuyển thành các vector. Sau đó RNN sẽ xử lý từng Vector một.

<img src="images/11.1_rnn_processing_sequnce.gif" style="width:100%;height:100%;">

Trong quá trình xử lý, cấu trúc RNN chuyển trạng thái ẩn ở bước $h_{t-1}$ thành đầu vào của bước thứ $t$  và cùng với đầu vào $x_t$. Trạng thái ẩn hoạt động như bộ nhớ trong mạng. Chính vì thế nó giúp lưu trữ các thông tin phía trước.

<img src="images/11.2_rnn.gif" style="width:70%;height:70%;">

Bây giờ, chúng ta hãy xét 1 Cell của RNN để xem bên trong trạng thái ẩn tính toán như thế nào. Đầu tiên, đầu vào và trạng thái ẩn ở bước phía trước kết hợp với nhau tạo thành 1 vector. Vector này hiện có thông tin đầu vào và "bộ nhớ" của phía trước. Sau đó vector này đi qua hàm kích hoạt `tanh` và đầu ra là trạng thái ẩn mới (bộ nhớ mới có thêm thông tin vừa được nhập vào).

<img src="images/12.3_rnn.gif" style="width:70%;height:70%;">


### 3. Mô hình LSTM

Đối với mô hình LSTM cũng có kiến trúc tương tự như với RNN, nhưng các modun bên trong không chỉ có 1 hàm duy nhất. Mà nó bao gồm 4 layers tương tác với nhau:

<img src="images/12_LSTM3-chain.png" style="width:70%;height:70%;">



Chúng ta sẽ cùng nhau đi xem xét từng thành phần bên trong mô hình này, nhưng trước hết, hãy cùng nhau xem qua một số ký hiệu: 

<img src="images/13_LSTM2-notation.png" style="width:70%;height:70%;">



Trong mô hình bên trên, mỗi đường kẻ đen là các vector từ đầu ra của một nút đến đầu vào của một nút khác. Các hình tròn màu hồng đại diện cho các phép toán như: nhân, cộng vector. Trong khi các hình chữ nhật màu vàng được sử dụng để học trong các lớp mạng neural. Kí hiệu 2 đường kẻ hợp nhau ám chỉ việc kết hợp, còn 2 đường kẻ tách nhau ra ám chỉ việc sao chép.

LSTM gồm 4 thành phần chính: Cell State, Input Gate, Output Gate và Forget Gate. Bây giờ chúng ta sẽ cùng đi xem xét từng thành phần một.

#### 3.1 Nhắc lại hàm kích hoạt Tanh và Sigmoid 

#### Activation Tanh 

Hàm kích hoạt Tanh được sử dụng để điều chỉnh các giá trị khi đi qua mạng. Giá trị của hàm Tanh nằm trong khoảng từ -1 đến 1.

<img src="images/13_tanh.gif" style="width:50%;height:50%;">

Khi các Vector đi qua mạng Neural, nó trải qua nhiều biến động bởi các hàm toán học. 
Hãy thử giả sử nhân liên tiếp các giá trị lên 3 lần thì bạn sẽ thấy các giá trị dần dần tăng lên rất nhanh, khiến các giá trị đầu ra có thể tiến ra vô cùng.

<img src="images/13_without_tanh.gif" style="width:100%;height:100%;">

Mặt khác, khi sử dụng các hàm kích hoạt nói chung, trong ví dụ này với hàm `tanh`, các bạn có thể thấy giới hạn của các giá trị nằm trong khoảng giới hạn của hàm `tanh`: -1 đến 1.

<img src="images/13_with_tanh.gif" style="width:100%;height:100%;">

#### Activation Sigmoid

Hàm `Sigmoid` nói chung cũng mang chức năng tương tự với hàm `Tanh`. Tuy nhiên, thay vì khoảng giá trị (-1, 1) như của `Tanh` thì hàm `Sigmoid` mang giá trị (0, 1). Hàm `Sigmoid` rất hữu ích cho sử dụng cho việc "quên" hay "cập nhật" giá trị vì bất kỳ số nào nhân với 0 đều ra 0, còn khi nhân với 1 thì nó sẽ dữ lại các giá trị.

<img src="images/13_sigmoid.gif" style="width:50%;height:50%;">


#### 3.2 Cell State

Chìa khoá của LSTM chính là Cell State, đường chạy phía trên cùng trong hình ảnh minh hoạ bên dưới.

Cell State là một dạng giống như băng truyền. Nó chạy xuyên suốt qua toàn bộ chuỗi, và chỉ tương tác tuyến tính đôi chút (nhân hay cộng vector). Vì vậy mà thông tin truyền đi không bị thay đổi quá nhiều.

<img src="images/14_LSTM3-C-line.png" style="width:70%;height:70%;">

LSTM có khả năng kiểm soát việc thông tin được thêm vào hoặc bỏ ra thông qua các "cổng" (gates). Các gates được tạo thành từ đầu vào là $x, h$ qua hàm sigmoid và kết hợp bởi tích Hadamard.

<img src="images/14_LSTM3-gate.png" style="width:10%;height:10%;">

Hàm `sigmoid` có giá trị từ 0 đến 1 quyết định độ "mạnh / yếu" của thông tin đi qua.

#### 3.3 Forget Gate 

<img src="images/15_LSTM3-focus-f.png" style="width:70%;height:70%;">

Bước đầu tiên trong LSTM là Forget Gate (cổng quên): quyết định bao nhiêu thông tin cũ $C_{t-1}$ phải bỏ đi.
Quyết định này được đưa ra bởi hàm `sigmoid`. Đầu vào sẽ gồm trạng thái ẩn của bước trước $h_{t-1}$ và $x_t$ và kết quả đầu ra có giá trị từ 0 đến 1 cho mỗi số trong Cell State. 


Ảnh minh hoạ trực quan:
<img src="images/15_focus.gif" style="width:50%;height:50%;">


#### 3.4 Input Gate 

<img src="images/16_LSTM3-focus-i.png" style="width:70%;height:70%;">

Input Gate (cổng vào) quyết định thông tin mới nào sẽ được lưu vào Cell. Bước này gồm 2 phần: tính input gate và giá trị  $\tilde{C_t}$ sẽ được thêm vào $C_{t-1}$.

Đầu tiên, chúng ta sẽ chuyển trạng thái ẩn $h_{t-1}$ và đầu vào hiện tại $x_t$ vào hàm `Sigmoid` để quyết định cập nhật giá trị nào. Với giá trị từ 0 đến 1 trong hàm `Sigmoid` thì với giá trị 0 - thông tin không quan trọng nên không cập nhật; với giá trị 1 - thông tin quan trọng nên giữ lại.

Đồng thời, ta cũng đưa $h_{t-1}$ và $x_t$ qua hàm `tanh` để điều chỉnh giá trị trong khoảng -1 đến 1. Sau đó kết hợp với đầu ra hàm `sigmoid` thông qua việc nhân từng phần tử Vector sẽ quyết định giá trị nào là quan trọng.

Ảnh minh hoạ trực quan:

<img src="images/16_input_cell.gif" style="width:50%;height:50%;">


#### 3.5 Update Cell State 

<img src="images/17_LSTM3-focus-C.png" style="width:70%;height:70%;">

Cập nhật giá trị của Cell $(C)$ dựa vào giá trị cũ của $C_{t-1}$ và thông tin mới sẽ được thêm vào.

Trong bước này sẽ thực hiện nhân trạng thái cũ $C_{t-1}$ với $f_t$. Sau đó cộng thêm với $i_t * \tilde{C_t}$. Trạng thái mới $C_t$ thu được này phụ thuộc vào việc quyết định quên những gì và cập nhật thêm các giá trị mới.

Ảnh minh hoạ trực quan:

<img src="images/17_update_cell_state.gif" style="width:50%;height:50%;">

#### 3.6 Output Gate

<img src="images/18_LSTM3-focus-o.png" style="width:70%;height:70%;">

Cuối cùng, chúng ta sẽ có Output Gate (cổng ra). Output Gate quyết định trạng thái ẩn tiếp theo là gì.

Đầu tiên ta đưa trạng thái ẩn $h_{t-1}$ và $x_t$ qua hàm `Sigmoid` để quyết định phần nào của Cell State muốn xuất ra. Và đưa trạng thái $C_t$ qua hàm `tanh` để các giá trị của $C_t$ từ -1 đến 1. Cuối cùng, nhân 2 giá trị này với nhau, ta được đầu ra.

Ảnh minh hoạ trực quan:
<img src="images/18_output_gate.gif" style="width:50%;height:50%;">


#### 3.7 Code minh hoạ các cổng

In [62]:
import numpy as np
from scipy.special import expit as sigmoid

def forget_layer(Wf, xt, Uf, ht_prev, bf):
    forget_eventx = np.dot(Wf, xt)
    forget_hidden = np.dot(Uf, ht_prev)
    return sigmoid(forget_hidden + forget_eventx + bf)

def input_gate(Wi, xt, Ui, ht_prev, bi, Wc, Uc, bc):
    ignore_eventx = np.dot(Wf, xt)
    ignore_hidden = np.dot(Uf, ht_prev)
    learn_eventx = np.dot(Wc, xt)
    learn_hidden = np.dot(Uc, ht_prev)
    return np.multiply(sigmoid(ignore_eventx + ignore_hidden + bi), np.tanh(learn_eventx + learn_hidden + bc))

def update_cell_state(forget_layer_output, input_gate_output, prev_cell_state):
    return np.multiply(forget_layer_output, prev_cell_state) + input_gate_output

def output_gate(Wo, xt, Uo, ht_prev, bo, cell_state):
    out_eventx = np.dot(Wo, xt)
    out_hidden = np.dot(Uo, ht_prev)
    return np.multiply(sigmoid(out_eventx + out_hidden + bo), cell_state)

Xét bài toán tính giá trị $h_1$ với đầu vào $x_1 = [2, 3, -1]$

Các parameters của mạng:

In [63]:
Wf = np.asarray([[1, -2, 5], [5, 5, 0]])
Uf = np.asarray([[1, 5], [2, 1]])

Wi = np.asarray([[0, 2, 5], [3, -4, 1]])
Ui = np.asarray([[-1, -5], [2, 1]])

Wo = np.asarray([[1, 2, 1], [0, -1, 3]])
Uo = np.asarray([[2, 4], [0, 3]])

Wc = np.asarray([[-2, 0, 1], [5, 1, 8]])
Uc = np.asarray([[1, -1], [0, -2]])

x = np.asarray([2, 3, -1])

bf = bi = bo = bc = np.asarray([5, 2])

In [64]:

ct = [0, 0]
ht = [0, 0]

def LSTMCELL(prev_ct, prev_ht, input_x):
    forget_layer_out = forget_layer(Wf, input_x, Uf, prev_ht, bf)
    intput_gate_out = input_gate(Wi, input_x, Ui, prev_ht, bi, Wc, Uc, bc)
    cell_state = update_cell_state(forget_layer_out, intput_gate_out, prev_ct)
    output = output_gate(Wo, input_x, Uo, prev_ht, bo, cell_state)
    
    return cell_state, output

cell_state_t, output_h_t = LSTMCELL(ct, ht, x)

print('New cell state:', cell_state_t)
print('Output: ', output_h_t)
    

New cell state: [0.         0.99999834]
Output:  [0.         0.01798618]


### Tổng kết mô hình LSTM 

<img src="images/Sumary.png">


### 4. Các biến thể của LSTM 

#### 4.1 Biến thể LSTM 1

Một dạng mô hình LSTM phổ biến được giới thiệu bởi [Gers & Schmidhuber (2000)](ftp://ftp.idsia.ch/pub/juergen/TimeCount-IJCNN2000.pdf).
Trong mô hình này, giá trị của $C$ được sử dụng để tính giá trị của các Gates. Tác giả đã thêm các đường kết nối “peephole connections”.

<img src="images/19_LSTM3-var-peepholes.png" style="width:70%;height:70%;">

Hình bên trên mô tả các đường “peephole connections” được thêm vào cổng có hàm kích hoạt `sigmoid`, nhưng cũng có một số bài báo khác chỉ thêm vào 1 hay 2 cổng.

#### 4.2 Biến thể LSTM 2

Trong biến thể này, nối 2 cổng quyết định giữ / bỏ dữ liệu với nhau thay vì phải tách ra thành các quyết định thêm thông tin vào hay bỏ thông tin đi.

<img src="images/20_LSTM3-var-tied.png" style="width:70%;height:70%;">

#### 4.3 Mô hình Gated Recurrent Unit (GRU)

Một biến thể khá ấn tượng của LSTM là GRU được giới thiệu bởi [Cho, et al. (2014)](http://arxiv.org/pdf/1406.1078v3.pdf). Nó kết hợp các cổng "quên" và cổng đầu vào thành một cổng cập nhật duy nhất. Nó cũng hợp các Cell State và trạng thái ẩn với nhau tạo ra một thay đổi khác.

<img src="images/21_LSTM3-var-GRU.png" style="width:70%;height:70%;">

Trên đây chỉ là một số mô hình biến thể của LSTM tiêu biểu. Ngoài ra vẫn còn các mô hình khác, các bạn có thể tìm kiếm trên Google về các biến thể này để biết thêm.

### Tài liệu tham khảo 

[1] [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/)

[2] [Illustrated Guide to LSTM’s and GRU’s: A step by step explanation](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)

[3] [To understand LSTM architecture, code a forward pass with just NumPy](https://towardsdatascience.com/the-lstm-reference-card-6163ca98ae87)